![Image](https://docs.dask.org/en/latest/_static/images/dask-horizontal-white.svg)

To Lean More about Dask [Follow Here](https://dask.org/) 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Pandas Dataframe

In [ ]:
%%time
train= pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
       usecols=[1, 2, 3,4,7,8,9], dtype={'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16','content_type_id': 'int8',
        'answered_correctly':'int8','prior_question_elapsed_time': 'float32','prior_question_had_explanation': 'boolean'}
              )

In [ ]:
%%time
train.head()

In [ ]:
%%time
results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']

It takes 14.3 seconds to compute the above cell, Lets see how dask is performing on that 

# Dask Dataframe

In [ ]:
import dask.dataframe as dd

In [ ]:
%%time
train_dask = dd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv",
       usecols=[1, 2, 3,4,7,8,9], dtype={'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16','content_type_id': 'int8',
        'answered_correctly':'int8','prior_question_elapsed_time': 'float32','prior_question_had_explanation': 'boolean'})

### That was a huge time difference between pandas and DASK

In [ ]:
%%time
train_dask.head(npartitions=-1)

> usually dask read the csv files in a chunk of partition so to take up all the partition we must use the *nparameter=-1* argument .

In [ ]:
%%time
train_dask.head()

Yes it is faster than previous one, as it is printing from one partition

## Let's see how many partitions are there in the dask dataframe


In [ ]:
train_dask.npartitions

### There are toally 92 partitions dask has created on this dataset

In [ ]:
%%time
train_dask

Printing the entire dataframe takes less than 10 micro seconds

In [ ]:
%%time
results_u = train_dask[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']

voila It takes just 19.7 ms to compute the above cell, Which is actually great to see 

In [ ]:
import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
import cudf

In [ ]:
from time import time
from contextlib import contextmanager

In [ ]:


%%time

data = cudf.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv')

print("Train size:", data.shape)

